In [1]:
import requests
import re
import time
import string
import random
import math
import pandas as pd
from urllib.parse import urlparse
from collections import Counter
from bs4 import BeautifulSoup
from pymystem3 import Mystem

# Задание №1

In [2]:
def crawler(urls):
    links_tree = set()
    for url in urls:
        if len(texts) >= 10:
            return
    
        if url in visited:
            continue
        visited.add(url)
        
        try:
            #получаем контент страницы и декодируем его
            html = requests.get(url).content.decode()
        except:
            time.sleep(1)
            continue
        
        #парсинг страницы
        soup = BeautifulSoup(html)
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text(' ').strip().replace('\xa0', ' ').replace('\n', '').replace('\r', '').replace('\t', '')
        # re.sub() для замены всех ' +' на ' '
        text = re.sub(' +', ' ', text)
    
        if len(text.split(' ')) >= 10:
            texts[url] = text
            
        parsed = urlparse(url)
        base = f"{parsed.scheme}://{parsed.netloc}"
        # re.findall() - найти все вхождения
        links = re.findall('''<a\s+(?:[^>]*?\s+)?href="([^"]*)"''', html) 
        
        for i, link in enumerate(links):    
            if not urlparse(link).netloc:    
                link_with_base = base + link    
                links[i] = link_with_base
                
        links_tree.update(set(filter(lambda x: 'mailto' not in x, links)))
    
    crawler(links_tree)

In [3]:
texts = {}
visited = set()
start = 'https://m.habr.com/ru/all/'

crawler([start])

In [4]:
index = {i:k for i, k in enumerate(texts)} 

In [5]:
with open('index.txt', 'w') as f:
    for i in index:
        f.write(f"{i}: {index[i]}\n")

# Задание №2

In [6]:
from openpyxl import load_workbook
import pandas as pd
import os, pymorphy2

In [7]:
words_all = {}
lemma_words = {}
lemma_normal_forms = {}

In [8]:
def get_normal_forms(word_list):
    # получить список нормальных форм
    morph, lst = pymorphy2.MorphAnalyzer(), []
    return [morph.parse(word)[0].normal_form
            for word in word_list]

In [9]:
def get_sparsed_list(word_list):
    # получить список всех словоформ
    morph, lst = pymorphy2.MorphAnalyzer(), []
    for word in word_list:
        lemma = ""
        for x in morph.parse(word)[0].lexeme:
            lemma+=x.word + " "
        lst.append(f"{word} {lemma}\n")
    return lst

In [10]:
texts_lemmatized = {}

for url in texts:
    text = texts[url]
    # оставляем только слова
    text = re.sub(r'[^a-zA-Zа-яА-Я]+', ' ', text)
   
    words = text.split(' ')
    words_all[url]=words
    
    lemma_words[url]=get_sparsed_list(words_all[url])  
    lemma_normal_forms[url]=get_normal_forms(words_all[url])  

In [11]:
with open('words.txt', 'w') as f:
    for url in words_all:
        for i, score in enumerate(words_all[url]):
            f.write(f"{score}\n")

In [12]:
with open('lemma_words.txt', 'w') as f:
    for url in lemma_words:
        for i, score in enumerate(lemma_words[url]):
            f.write(f"{score}\n")

# Задание №3

In [13]:
docs = [index[k] for k in index]

In [15]:
 # Создание инвертированного списка терминов (индекс)
reverse_index = {}

for url in lemma_normal_forms:
     # пройдемся по списку лемматизированных токенов
    for word in lemma_normal_forms[url]:
        if not word in reverse_index:
             # если слово не входит в reverse_index, то добавляем его туда
            reverse_index[word] = set([url])
        else:
            reverse_index[word].update([url])

In [16]:
# булев поиск по построенному индексу

logic = {
    '&': lambda x,y: x & y,
    '|': lambda x,y: x | y,
    '!': lambda x: set([doc for doc in docs if doc not in x])
}

def find(query):
    query = query.replace('(', ' ( ').replace(')', ' ) ').replace('!','! ').split()
    
    # слова, которые должны присутствовать на странице
    indexes_words = [i for i,x in enumerate(query) if not x in ['!', '&', '|']]
    for i in indexes_words:
        query[i] = reverse_index[query[i]]
    
    # слова, которые не должны присутствовать на странице
    indexes_not = [i for i,x in enumerate(query) if x == '!']
    for i in indexes_not:
        x = query[i+1]
        query[i+1] = logic['!'](x)
        query.pop(i)
    
    # фразы, которые должны присутствовать на странице
    indexes_and = [i for i,x in enumerate(query) if x == '&']
    for i in indexes_and:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['&'](x,y)
        # list.pop([i]) - Возвращает элемент [на указанной позиции], удаляя его из списка.
        query.pop(i)
        query.pop(i)
    
    # хотя бы одно из указанных тут слов должно присутствовать на странице
    indexes_or = [i for i,x in enumerate(query) if x == '|']
    for i in indexes_or:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['|'](x,y)
        query.pop(i)
        query.pop(i)
    
    return query[0]

In [82]:
# random.choice() - возвращает случайный элемент из указанной последовательности
x1 = random.choice(list(reverse_index.keys()))
x2 = random.choice(list(reverse_index.keys()))
x3 = random.choice(list(reverse_index.keys()))

In [83]:
query = f'{x1} | {x2} & !{x3}'
print(query)

python | невозврат & !пиксель


In [84]:
find(query)

{'https://m.habr.com/ru/all/',
 'https://m.habr.com/ru/news/',
 'https://m.habr.com/ru/post/543688/'}

# Задание №4

TF термина а = (Количество раз, когда термин а встретился в тексте / количество всех слов в тексте)

In [20]:
tfs = {}

for word in reverse_index:
    for doc in docs:
        tokens = texts[doc].split()
        tf = tokens.count(word) / len(tokens)
        if not word in tfs:
            tfs[word] = {}
        tfs[word][doc] = tf

In [59]:
pd.DataFrame(tfs).T

,https://m.habr.com/ru/all/,https://m.habr.com/ru/news/,https://m.habr.com/ru/users/honyaki/,https://m.habr.com/ru/docs/authors/codex/,https://tmtm.ru/services/corpblog/,https://m.habr.com/ru/post/543688/,https://m.habr.com/ru/company/selectel/profile/,https://m.habr.com/ru/company/mailru/profile/,https://m.habr.com/ru/post/546180/comments/,https://m.habr.com/ru/news/t/546166/comments/
всё,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
публикация,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
подряд,0.002532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
хабра,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
открыть,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
оставлять,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005263,0.005051
научно,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
популярный,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
космонавтика,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


IDF термина а = логарифм(Общее количество документов / Количество документов, в которых встречается термин а)

In [60]:
idfs = {}

for word in reverse_index:
    idfs[word] = math.log(len(docs) / len(reverse_index[word]))

In [61]:
pd.DataFrame(idfs, index = ['']).T

,
всё,0.000000
публикация,0.000000
подряд,2.302585
хабра,0.000000
открыть,0.105361
...,...
оставлять,1.609438
научно,2.302585
популярный,2.302585
космонавтика,2.302585


TF-IDF термина а = (TF термина а) * (IDF термина а)

In [62]:
tfidfs = {}

for word in tfs:
    for doc in tfs[word]:
        tfidf = tfs[word][doc] * idfs[word]
        if not word in tfidfs:
            tfidfs[word] = {}
        tfidfs[word][doc] = tfidf

In [63]:
pd.DataFrame(tfidfs).T

,https://m.habr.com/ru/all/,https://m.habr.com/ru/news/,https://m.habr.com/ru/users/honyaki/,https://m.habr.com/ru/docs/authors/codex/,https://tmtm.ru/services/corpblog/,https://m.habr.com/ru/post/543688/,https://m.habr.com/ru/company/selectel/profile/,https://m.habr.com/ru/company/mailru/profile/,https://m.habr.com/ru/post/546180/comments/,https://m.habr.com/ru/news/t/546166/comments/
всё,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
публикация,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
подряд,0.005829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
хабра,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
открыть,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
оставлять,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008471,0.008128
научно,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
популярный,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
космонавтика,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


# Задание 5

In [166]:
docs_length = {}

In [167]:
for row in pd.DataFrame(tfidfs).iterrows():
    sum_len = 0
    for word in row[1]:
        sum_len += word**2
    docs_length[row[0]] = math.sqrt(sum_len)

In [168]:
docs_length

{'https://m.habr.com/ru/all/': 0.015568175855670077,
 'https://m.habr.com/ru/news/': 0.019566530933460145,
 'https://m.habr.com/ru/users/honyaki/': 0.024816796047733063,
 'https://m.habr.com/ru/docs/authors/codex/': 0.046427184958068086,
 'https://tmtm.ru/services/corpblog/': 0.04058795035167561,
 'https://m.habr.com/ru/post/543688/': 0.022003911396354776,
 'https://m.habr.com/ru/company/selectel/profile/': 0.022923800925160706,
 'https://m.habr.com/ru/company/mailru/profile/': 0.027345089335982058,
 'https://m.habr.com/ru/post/546180/comments/': 0.028001835068165237,
 'https://m.habr.com/ru/news/t/546166/comments/': 0.018218445658959064}

In [169]:
query = "ожидать"

In [173]:
morph, lst = pymorphy2.MorphAnalyzer(), []
lemma = ""
for x in morph.parse(query)[0].lexeme:
    lemma+=x.word + " "

In [174]:
lemma

'ожидать ожидаю ожидаем ожидаешь ожидаете ожидает ожидают ожидал ожидала ожидало ожидали ожидай ожидайте ожидающий ожидающего ожидающему ожидающего ожидающий ожидающим ожидающем ожидающая ожидающей ожидающей ожидающую ожидающей ожидающею ожидающей ожидающее ожидающего ожидающему ожидающее ожидающим ожидающем ожидающие ожидающих ожидающим ожидающих ожидающие ожидающими ожидающих ожидавший ожидавшего ожидавшему ожидавшего ожидавший ожидавшим ожидавшем ожидавшая ожидавшей ожидавшей ожидавшую ожидавшей ожидавшею ожидавшей ожидавшее ожидавшего ожидавшему ожидавшее ожидавшим ожидавшем ожидавшие ожидавших ожидавшим ожидавших ожидавшие ожидавшими ожидавших ожидаемый ожидаемого ожидаемому ожидаемого ожидаемый ожидаемым ожидаемом ожидаемая ожидаемой ожидаемой ожидаемую ожидаемой ожидаемою ожидаемой ожидаемое ожидаемого ожидаемому ожидаемое ожидаемым ожидаемом ожидаемые ожидаемых ожидаемым ожидаемых ожидаемые ожидаемыми ожидаемых ожидая ожидав ожидавши ожидаем ожидаема ожидаемо ожидаемы '

In [175]:
for word in lemma.split():
    for doc in docs:
        tokens = texts[doc].split()
        tf = tokens.count(word) / len(tokens)
        if not word in tfs_query:
            tfs_query[word] = {}
        tfs_query[word][doc] = tf

In [176]:
tfs_query

{'ожидать': {'https://m.habr.com/ru/all/': 0.0,
  'https://m.habr.com/ru/news/': 0.0,
  'https://m.habr.com/ru/users/honyaki/': 0.0,
  'https://m.habr.com/ru/docs/authors/codex/': 0.0,
  'https://tmtm.ru/services/corpblog/': 0.0,
  'https://m.habr.com/ru/post/543688/': 0.0,
  'https://m.habr.com/ru/company/selectel/profile/': 0.0,
  'https://m.habr.com/ru/company/mailru/profile/': 0.0,
  'https://m.habr.com/ru/post/546180/comments/': 0.0,
  'https://m.habr.com/ru/news/t/546166/comments/': 0.0},
 'ожидаю': {'https://m.habr.com/ru/all/': 0.0,
  'https://m.habr.com/ru/news/': 0.0,
  'https://m.habr.com/ru/users/honyaki/': 0.0,
  'https://m.habr.com/ru/docs/authors/codex/': 0.0,
  'https://tmtm.ru/services/corpblog/': 0.0,
  'https://m.habr.com/ru/post/543688/': 0.0,
  'https://m.habr.com/ru/company/selectel/profile/': 0.0,
  'https://m.habr.com/ru/company/mailru/profile/': 0.0,
  'https://m.habr.com/ru/post/546180/comments/': 0.0,
  'https://m.habr.com/ru/news/t/546166/comments/': 0.0},
 

In [184]:
idfs_query = {}

for word in lemma.split():
    print(reverse_index)
    idfs_query[word] = math.log(len(docs) / len(reverse_index[word]))


{'всё': {'https://m.habr.com/ru/news/', 'https://m.habr.com/ru/all/', 'https://m.habr.com/ru/docs/authors/codex/', 'https://m.habr.com/ru/users/honyaki/', 'https://tmtm.ru/services/corpblog/', 'https://m.habr.com/ru/post/543688/', 'https://m.habr.com/ru/company/selectel/profile/', 'https://m.habr.com/ru/company/mailru/profile/', 'https://m.habr.com/ru/post/546180/comments/', 'https://m.habr.com/ru/news/t/546166/comments/'}, 'публикация': {'https://m.habr.com/ru/news/', 'https://m.habr.com/ru/all/', 'https://m.habr.com/ru/docs/authors/codex/', 'https://m.habr.com/ru/users/honyaki/', 'https://tmtm.ru/services/corpblog/', 'https://m.habr.com/ru/post/543688/', 'https://m.habr.com/ru/company/selectel/profile/', 'https://m.habr.com/ru/company/mailru/profile/', 'https://m.habr.com/ru/post/546180/comments/', 'https://m.habr.com/ru/news/t/546166/comments/'}, 'подряд': {'https://m.habr.com/ru/all/'}, 'хабра': {'https://m.habr.com/ru/news/', 'https://m.habr.com/ru/all/', 'https://m.habr.com/ru/do

KeyError: 'ожидаю'

In [178]:
idfs_query

{'ожидать': 1.6094379124341003}

In [179]:
for word in tfs_query:
    for doc in tfs_query[word]:
        tfidf = tfs_query[word][doc] * idfs_query[word]
        if not word in tfidfs_query:
            tfidfs_query[word] = {}
        tfidfs_query[word][doc] = tfidf

KeyError: 'ожидаю'

In [163]:
tfidfs_query

{'ожидание': {'https://m.habr.com/ru/all/': 0.0,
  'https://m.habr.com/ru/news/': 0.0,
  'https://m.habr.com/ru/users/honyaki/': 0.006787338754623371,
  'https://m.habr.com/ru/docs/authors/codex/': 0.0,
  'https://tmtm.ru/services/corpblog/': 0.0,
  'https://m.habr.com/ru/post/543688/': 0.0003840279680947842,
  'https://m.habr.com/ru/company/selectel/profile/': 0.002768249945239139,
  'https://m.habr.com/ru/company/mailru/profile/': 0.001877644942365072,
  'https://m.habr.com/ru/post/546180/comments/': 0.0,
  'https://m.habr.com/ru/news/t/546166/comments/': 0.0},
 'vs': {'https://m.habr.com/ru/all/': 0.0,
  'https://m.habr.com/ru/news/': 0.0,
  'https://m.habr.com/ru/users/honyaki/': 0.0007804482641320471,
  'https://m.habr.com/ru/docs/authors/codex/': 0.0,
  'https://tmtm.ru/services/corpblog/': 0.0,
  'https://m.habr.com/ru/post/543688/': 4.415780203597081e-05,
  'https://m.habr.com/ru/company/selectel/profile/': 0.0003183097149783273,
  'https://m.habr.com/ru/company/mailru/profile/

In [164]:
tfidfs_query_flat = pd.DataFrame(tfidfs_query).T.sum().to_dict()

In [165]:
sum_len = 0
for doc in tfidfs_query_flat:
    sum_len += tfidfs_query_flat[doc]**2
query_length = math.sqrt(sum_len)
 
cos_sim = {}
for doc in docs:
    for word in query.split(' '):
        cs = ((pd.Series(tfidfs_query_flat) * tfs[word][doc]).sum()) / (docs_length[doc] * query_length)
    cos_sim[doc] = cs
 
Counter(cos_sim).most_common(10)

[('https://m.habr.com/ru/all/', 0.0),
 ('https://m.habr.com/ru/news/', 0.0),
 ('https://m.habr.com/ru/users/honyaki/', 0.0),
 ('https://m.habr.com/ru/docs/authors/codex/', 0.0),
 ('https://tmtm.ru/services/corpblog/', 0.0),
 ('https://m.habr.com/ru/post/543688/', 0.0),
 ('https://m.habr.com/ru/company/selectel/profile/', 0.0),
 ('https://m.habr.com/ru/company/mailru/profile/', 0.0),
 ('https://m.habr.com/ru/post/546180/comments/', 0.0),
 ('https://m.habr.com/ru/news/t/546166/comments/', 0.0)]